In [4]:
import matplotlib.pyplot as plt
from skimage import io, util
import cv2
import numpy as np
from scipy.ndimage import median_filter

# Load the uploaded image (color)
image_path = "sattelite.png"
color_image = io.imread(image_path)

# Convert to grayscale for processing
gray_image = util.img_as_ubyte(np.mean(color_image, axis=2) / 255.0)


# Method 1: Median filter (scipy)
median_denoised = median_filter(gray_image, size=3)

# Method 2: Morphological closing (OpenCV)
kernel = np.ones((3, 3), np.uint8)
morph_close = cv2.morphologyEx(gray_image, cv2.MORPH_CLOSE, kernel)

# Method 3: Horizontal median filter
horizontal_median = median_filter(gray_image, size=(1, 5))

# Method 4: Bilateral filter
bilateral = cv2.bilateralFilter(gray_image, d=9, sigmaColor=75, sigmaSpace=75)

# Stack grayscale back into color (naive approach using original color as reference)
def reapply_color(gray, reference_color):
    ratio = gray / np.maximum(np.mean(reference_color, axis=2), 1)
    recolored = np.clip(reference_color * ratio[..., None], 0, 255).astype(np.uint8)
    return recolored

# Generate recolored versions
results = {
    "original": color_image,
    "median": reapply_color(median_denoised, color_image),
    "morph_close": reapply_color(morph_close, color_image),
    "horizontal_median": reapply_color(horizontal_median, color_image),
    "bilateral": reapply_color(bilateral, color_image),
}

# Save each result
output_paths = {}
for name, img in results.items():
    out_path = f"denoised_{name}.png"
    io.imsave(out_path, img)
    output_paths[name] = out_path

output_paths


{'original': 'denoised_original.png',
 'median': 'denoised_median.png',
 'morph_close': 'denoised_morph_close.png',
 'horizontal_median': 'denoised_horizontal_median.png',
 'bilateral': 'denoised_bilateral.png'}